In [2]:
# coding: utf-8

# In[1]:
import keras
import os, shutil
from keras.callbacks import ModelCheckpoint
from tensorflow.keras .callbacks import TensorBoard
from tensorflow.keras  import regularizers
from tensorflow.keras import layers
from tensorflow.keras import models
from PIL import ImageFile
from tensorflow import keras

Using TensorFlow backend.


In [3]:
original_women_dir = './gender/Female'
base_dir = './gender/categories'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


train_women_dir = os.path.join(train_dir, 'women')
validation_women_dir = os.path.join(validation_dir, 'women')
test_women_dir = os.path.join(test_dir, 'women')


train_men_dir = os.path.join(train_dir, 'men')
validation_men_dir = os.path.join(validation_dir, 'men')
test_men_dir = os.path.join(test_dir, 'men')

In [4]:
print('total training male images:', len(os.listdir(train_men_dir)))
print('total training female images:', len(os.listdir(train_women_dir)))
print('total validation male images:', len(os.listdir(validation_men_dir)))
print('total validation female images:', len(os.listdir(validation_women_dir)))
print('total test male images:', len(os.listdir(test_men_dir)))
print('total test women images:', len(os.listdir(test_women_dir)))

total training male images: 700
total training female images: 679
total validation male images: 50
total validation female images: 57
total test male images: 246
total test women images: 251


In [160]:
# parameters should not exceed number of datapoints in training per label by a factor of 10.
# small dataste NN was able to memorize was not able to generalize
# model can capture variations
# smaller means number of parameters 
# increasing layers m decrease parameters
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 225, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


# In[5]:

model.summary()


from tensorflow.keras import optimizers

# sgd = optimizers.SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = optimizers.RMSprop(lr=1e-3, decay=1e-6)

model.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 148, 223, 32)      896       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 74, 111, 32)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 72, 109, 32)       9248      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 36, 54, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 34, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 17, 26, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 28288)             0         
__________

In [161]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,                    
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
# decreased batch size from 10 to 5
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x225
        target_size=(150, 225),
        batch_size=50,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 225),
        batch_size=5,
        class_mode='binary')

Found 1377 images belonging to 2 classes.
Found 105 images belonging to 2 classes.


In [ ]:
# val_loss_early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
#                               min_delta=0.1,
#                               patience=3,
#                               verbose=0, mode='auto')

# val_acc_early_stop = keras.callbacks.EarlyStopping(monitor='val_acc',
#                               min_delta=0.1,
#                               patience=3,
#                               verbose=0, mode='auto')

CHECKPOINT_FILE_PATH = 'checkpoint.{epoch:02d}.hdf5'

checkpoint = ModelCheckpoint(CHECKPOINT_FILE_PATH,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',
                             period=1)

ImageFile.LOAD_TRUNCATED_IMAGES = True

# Improvement after changing optimizer: https://keras.io/optimizers/
# Added SGD optimizer
# https://stackoverflow.com/questions/37213388/keras-accuracy-does-not-change

# board = TensorBoard(log_dir='./output',
#                             histogram_freq=0,
#                             batch_size=16,
#                             write_graph=True,
#                             write_grads=False,
#                             write_images=False,
#                             embeddings_layer_names=None,
#                             embeddings_metadata=None,
#                             embeddings_data=None)

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=100,
      shuffle=True,
      callbacks=[checkpoint]
)

Epoch 1/30
 99/100 [============================>.] - ETA: 4s - loss: 0.1539 - acc: 0.9458
Epoch 00001: val_acc improved from -inf to 0.57200, saving model to checkpoint.01.hdf5
100/100 [==============================] - 496s 5s/step - loss: 0.1535 - acc: 0.9459 - val_loss: 1.6493 - val_acc: 0.5720
Epoch 2/30
 64/100 [==================>...........] - ETA: 2:05 - loss: 0.1335 - acc: 0.9510

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()



model.save('thesatorialist.h5')

